In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests
import json
from selenium.webdriver import Chrome
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time
from datetime import datetime

In [2]:
driver = Chrome(executable_path='/Users/chrisjohnson/Downloads/chromedriver')
url = "https://renaissance-hotels.marriott.com/locations-list-view"
driver.get(url)
time.sleep(30)
# viewhotels_button = driver.find_elements_by_xpath('/html/body/section/pacsys-animation[3]/pacsys-directory/div[3]/button')[0]
# viewhotels_button.click()
# northamhotels_button = driver.find_elements_by_xpath('/html/body/section/pacsys-animation[3]/pacsys-directory/div[2]/pacsys-directory-listings/div/div[5]/a')[0]
# northamhotels_button.click()

In [3]:
page_source = driver.page_source
soup = BeautifulSoup(page_source, 'lxml')

In [4]:
stuff = soup.find_all('li', {'class', 'locations-list__hotel'})
# stuff
hotel_list = []
for x in stuff:  
    hotel = {}
    hotel['name'] = x.text.strip()
    url = x.find('a').attrs['href']        
    hotel['url'] = 'https://renaissance-hotels.marriott.com/' + url
    hotel_list.append(hotel)
hotelsurl_df = pd.DataFrame(hotel_list)
hotelsurl_df

,name,url
0,"Tlemcen, Renaissance Tlemcen Hotel",https://renaissance-hotels.marriott.com/renais...
1,"Cairo, Renaissance Cairo Mirage City Hotel",https://renaissance-hotels.marriott.com/renais...
2,"Sharm El Sheikh, Renaissance Sharm El Sheikh G...",https://renaissance-hotels.marriott.com/renais...
3,"Beijing, Renaissance Beijing Capital Hotel",https://renaissance-hotels.marriott.com/renais...
4,"Beijing, Renaissance Beijing Wangfujing Hotel",https://renaissance-hotels.marriott.com/renais...
...,...,...
166,"Fort Worth, The Worthington Renaissance Fort W...",https://renaissance-hotels.marriott.com/worthi...
167,"Richardson, Renaissance Dallas Richardson Hotel",https://renaissance-hotels.marriott.com/renais...
168,"Arlington, Renaissance Arlington Capital View ...",https://renaissance-hotels.marriott.com/renais...
169,"Portsmouth, Renaissance Portsmouth-Norfolk Wat...",https://renaissance-hotels.marriott.com/renais...


In [5]:
%%time

hotel_list = []
counter = 0
for index, row in hotelsurl_df.iterrows():
    name = row['name']
    url = row['url']
    try:
        res = requests.get(url)
        soup = BeautifulSoup(res.content, 'lxml')
        stuff = soup.find('div', {'class' : 'l-container l-header-container'})
#             hotel_list = []
        hotel = {}
        try:
            hotel['brand'] = 'Renaissance'
        except:
            hotel['brand'] = 'Renaissance'
        try:
            hotel['name'] = name
        except:
            hotel['name'] = name
        try:
            hotel['url'] = url
        except:
            hotel['url'] = url
        try:
            hotel['street'] = stuff.find_all('span')[2].text
        except:
            hotel['street'] = 'N/A'
        try:
            hotel['locality'] = stuff.find_all('span')[4].text
        except:
            hotel['locality'] = 'N/A'        
        try:
            hotel['state'] = stuff.find_all('span')[6].text
        except:
            hotel['state'] = 'N/A'            
        try:
            hotel['postalcode'] = stuff.find_all('span')[7].text
        except:
            hotel['postalcode'] = 'N/A'            
        try:
            hotel['coordinate1'] = stuff.find_all('span')[25].text
        except:
            hotel['coordinate1'] = 'N/A'            
        try:
            hotel['coordinate2'] = stuff.find_all('span')[26].text
        except:
            hotel['coordinate2'] = 'N/A'
        try:
            hotel['coordinate3'] = stuff.find_all('span')[27].text
        except:
            hotel['coordinate3'] = 'N/A'            
        try:
            hotel['coordinate4'] = stuff.find_all('span')[28].text
        except:
            hotel['coordinate4'] = 'N/A'            
        
        desc = soup.find_all('p', {'class' : 'l-margin-top-none l-overflow-hidden t-collapsed-height'})
        try:
            hotel['description'] = str(desc).replace('<p class="l-margin-top-none l-overflow-hidden t-collapsed-height">','').replace('</p>', '')
        except:
            hotel['description'] = 'N/A'
        
        overallratings = stuff = soup.find('div', {'class' : 'l-overall-rating-container'})
        try:
            hotel['overall_rating'] = ratings.find_all('div')[0].text.strip()[:4]
        except:
            hotel['overall_rating'] = 'N/A'
        
        specratings = stuff = soup.find('div', {'class' : 'l-secondary-ratings-container l-clear l-margin-bottom-quarter l-m-margin-bottom-none'})
        try:
            hotel['cleanliness_rating'] = specratings.find_all('div')[2].text.strip()[:4]
        except:
            hotel['cleanliness_rating'] = 'N/A'
        try:
            hotel['service_rating'] = specratings.find_all('div')[7].text.strip()[:4]
        except:
            hotel['service_rating'] = 'N/A'
            
        hotel_list.append(hotel)
        print(hotel)
        print()
        print(datetime.now())
        print('='*80)
        print('='*80)
        print()
#         driver.close()
#         driver.switch_to.window(original_window)
#         time.sleep(1)

        counter += 1
        print(f'Collected {counter} hotels.')
    except:
        hotel = {}
        hotel['name'] = name
        hotel['url'] = url
        hotel_list.append(hotel)
        print(f"Number {x} didn't work at {datetime.now()}")
        continue
hotel_df = pd.DataFrame(hotel_list)
hotel_df       

{'brand': 'Renaissance', 'name': 'Tlemcen, Renaissance Tlemcen Hotel', 'url': 'https://renaissance-hotels.marriott.com/renaissance-tlemcen-hotel', 'street': 'Plateau Lalla Setti', 'locality': 'Tlemcen', 'state': '', 'postalcode': '13000', 'coordinate1': 'Reviews', 'coordinate2': 'https://www.marriott.com/hotels/maps/travel/algbr-renaissance-tlemcen-hotel/?directPageRequest=true', 'coordinate3': '34.867492', 'coordinate4': '-1.315989', 'description': '[ The Renaissance Tlemcen is a 5-star hotel in Tlemcen, Algeria, located on Plateau Lalla Setti and near Messali Hadj Airport and the Tlemcen Train Station. All 204 sleek and spacious guest rooms feature luxurious bedding, free Wi-Fi and modern workspace facilities to ensure ultimate comfort for both business and leisure travelers. A thrill of senses awaits you at our six innovative restaurants, including Arabesque Steakhouse and Grill, Al Bustan with its Mediterranean cuisine and Alcazar bar and lounge. An outdoor pool, fully-equipped fit

{'brand': 'Renaissance', 'name': 'Chengdu, Renaissance Chengdu Hotel', 'url': 'https://renaissance-hotels.marriott.com/renaissance-chengdu-hotel', 'street': 'N/A', 'locality': 'N/A', 'state': 'N/A', 'postalcode': 'N/A', 'coordinate1': 'N/A', 'coordinate2': 'N/A', 'coordinate3': 'N/A', 'coordinate4': 'N/A', 'description': '[]', 'overall_rating': 'N/A', 'cleanliness_rating': 'N/A', 'service_rating': 'N/A'}

2020-09-21 23:05:29.339610

Collected 6 hotels.
{'brand': 'Renaissance', 'name': 'Guiyang, Guizhou, Renaissance Guiyang Hotel, Guizhou', 'url': 'https://renaissance-hotels.marriott.com/renaissance-guiyang-hotel-guizhou', 'street': 'No. 8 Lincheng East Road, Jinyang New District', 'locality': 'Guiyang', 'state': '', 'postalcode': '550081', 'coordinate1': 'Reviews', 'coordinate2': 'https://www.marriott.com/hotels/maps/travel/kwebr-renaissance-guiyang-hotel/?directPageRequest=true', 'coordinate3': '26.6443', 'coordinate4': '106.624114', 'description': "[ Designed for Social Discoverers, 

{'brand': 'Renaissance', 'name': 'Nanjing, Renaissance Nanjing Olympic Center Hotel', 'url': 'https://renaissance-hotels.marriott.com/renaissance-nanjing-olympic-center-hotel', 'street': '139 Aoti Street, Jianye District,', 'locality': 'Nanjing', 'state': '', 'postalcode': '210019', 'coordinate1': 'Reviews', 'coordinate2': 'https://www.marriott.com/hotels/maps/travel/nkgbr-renaissance-nanjing-olympic-centre-hotel/?directPageRequest=true', 'coordinate3': '32.002821', 'coordinate4': '118.724667', 'description': '[ Discover one of the four great ancient capitals of China at Renaissance Nanjing Olympic Centre Hotel. Nanjing has a firm place in the country’s history as an important inland ports and is quickly securing its status for the future with exciting growth in the city. Set in the Jianye District’s Hexi business circle, our modern business hotel boasts a prime location near the International Expo Centre and multiple industrial parks. The neighborhood is also offers shopping, dining a

{'brand': 'Renaissance', 'name': 'Shanghai, Renaissance Shanghai Caohejing Hotel', 'url': 'https://renaissance-hotels.marriott.com/renaissance-shanghai-caohejing-hotel', 'street': 'No 397 Tianlin Road, Caohejing Hi-Tech Park', 'locality': 'Shanghai', 'state': '', 'postalcode': '200233', 'coordinate1': 'Reviews', 'coordinate2': 'https://www.marriott.com/hotels/maps/travel/shaca-renaissance-shanghai-caohejing-hotel/?directPageRequest=true', 'coordinate3': '31.1653782', 'coordinate4': '121.3969195', 'description': "[ Renaissance Shanghai Caohejing Hotel welcomes you to China with sophisticated style and award-winning service. You'll find our hotel in the vibrant Xuhui district, within the Caohejing New Technology Development Zone; the National Exhibition and Convention Center (NECC) is close by, as is the Hongqiao business district and the ancient town of Qibao. Eclectically designed yet functional rooms and suites offer touches of whimsy and perks like complimentary high-speed Wi-Fi and 

{'brand': 'Renaissance', 'name': 'Suzhou Industrial Park, Renaissance Suzhou Hotel', 'url': 'https://renaissance-hotels.marriott.com/renaissance-suzhou-hotel', 'street': '229 Suzhou Avenue West, Suzhou Industrial Park', 'locality': 'Suzhou', 'state': '', 'postalcode': '215021', 'coordinate1': 'Reviews', 'coordinate2': 'https://www.marriott.com/hotels/maps/travel/szvbr-renaissance-suzhou-hotel/?directPageRequest=true', 'coordinate3': '31.3139917', 'coordinate4': '120.6645144', 'description': "[ Charmingly designed and conveniently located, Renaissance Suzhou Hotel provides an unforgettable backdrop for your next adventure. You'll find us ideally placed within Suzhou Industrial Park, moments away from Jinji Lake; the subway is just a few minutes away on foot, making it easy for Social Discoverers to explore. Settle into tranquil, contemporary rooms with pillowtop beds, marble bathrooms, ergonomic work areas with high-speed Wi-Fi and superb views over Suzhou Central Park. Sample Chinese a

...

{'brand': 'Renaissance', 'name': 'Columbus, Renaissance Columbus Westerville Hotel', 'url': 'https://renaissance-hotels.marriott.com/renaissance-columbus-westerville-hotel', 'street': '409 Altair Parkway', 'locality': 'Westerville', 'state': 'Ohio', 'postalcode': '43082', 'coordinate1': 'https://www.marriott.com/hotels/maps/travel/cmhbw-renaissance-columbus-westerville-polaris-hotel/?directPageRequest=true', 'coordinate2': '40.143236', 'coordinate3': '-82.942869', 'coordinate4': 'N/A', 'description': '[ Experience the Renaissance Columbus Westerville hotel.  Whether you are traveling to Columbus for work or for fun, the Renaissance Columbus Westerville-Polaris hotel is the ready to help you create lasting memories.   Embrace total comfort in spacious rooms and suites, well-appointed with contemporary amenities and deluxe bedding.  Recharge and reconnect with a specialty cocktail at our lounge and restaurant with sweeping views of our natural surroundings.  Ideally located in Westervill

{'brand': 'Renaissance', 'name': 'Allentown, Renaissance Allentown Hotel', 'url': 'https://renaissance-hotels.marriott.com/renaissance-allentown-hotel', 'street': 'N/A', 'locality': 'N/A', 'state': 'N/A', 'postalcode': 'N/A', 'coordinate1': 'N/A', 'coordinate2': 'N/A', 'coordinate3': 'N/A', 'coordinate4': 'N/A', 'description': '[]', 'overall_rating': 'N/A', 'cleanliness_rating': 'N/A', 'service_rating': 'N/A'}

2020-09-21 23:12:15.741602

Collected 156 hotels.
{'brand': 'Renaissance', 'name': 'Philadelphia, Renaissance Philadelphia Airport Hotel', 'url': 'https://renaissance-hotels.marriott.com/renaissance-philadelphia-airport-hotel', 'street': '500 Stevens Drive', 'locality': 'Philadelphia', 'state': 'Pennsylvania', 'postalcode': '19113', 'coordinate1': '39.873097', 'coordinate2': '-75.277209', 'coordinate3': 'N/A', 'coordinate4': 'N/A', 'description': "[ Relax and unwind at the newly renovated Renaissance Philadelphia Airport Hotel. With an unrivaled location less than a mile from Ph

{'brand': 'Renaissance', 'name': 'Nashville, Renaissance Nashville Hotel', 'url': 'https://renaissance-hotels.marriott.com/renaissance-nashville-hotel', 'street': '611 Commerce Street', 'locality': 'Nashville', 'state': 'Tennessee', 'postalcode': '37203', 'coordinate1': 'Reviews', 'coordinate2': 'https://www.marriott.com/hotels/maps/travel/bnash-renaissance-nashville-hotel/?directPageRequest=true', 'coordinate3': '36.160802', 'coordinate4': '-86.781142', 'description': "[ Recently renovated with inspiring features, Renaissance Nashville Hotel is a luxury haven in legendary Music City; we're situated in the heart of downtown, conveniently attached to Fifth + Broad and moments from attractions like Bridgestone Arena and Music Row. Recharge in luxurious guest rooms and suites with plush bedding, expansive workstations, marble bathrooms and striking views of downtown Nashville, Tennessee. Upgrade your stay in a deluxe suite, which boasts extra space, or book a Club-level hotel room for Lou

{'brand': 'Renaissance', 'name': 'Fort Worth, The Worthington Renaissance Fort Worth Hotel', 'url': 'https://renaissance-hotels.marriott.com/worthington-renaissance-fort-worth-hotel', 'street': '200 Main Street', 'locality': 'Fort Worth', 'state': 'Texas', 'postalcode': '76102', 'coordinate1': 'Reviews', 'coordinate2': 'https://www.marriott.com/hotels/maps/travel/dfwdt-the-worthington-renaissance-fort-worth-hotel/?directPageRequest=true', 'coordinate3': '32.756443', 'coordinate4': '-97.332436', 'description': "[ A historic fixture on the downtown landscape, The Worthington Renaissance Fort Worth Hotel offers contemporary comfort near celebrated attractions. Explore some of Fort Worth's top sights from our hotel, including the Kimbell Art Museum, Fort Worth Stockyards and Texas Christian University. Ignite your senses with delectable dishes and innovative cocktails at Toro Toro, our restaurant headed by world-renowned Chef Richard Sandoval. Achieve more than you ever thought possible in

,brand,name,url,street,locality,state,postalcode,coordinate1,coordinate2,coordinate3,coordinate4,description,overall_rating,cleanliness_rating,service_rating
0,Renaissance,"Tlemcen, Renaissance Tlemcen Hotel",https://renaissance-hotels.marriott.com/renais...,Plateau Lalla Setti,Tlemcen,,13000,Reviews,https://www.marriott.com/hotels/maps/travel/al...,34.867492,-1.315989,[ The Renaissance Tlemcen is a 5-star hotel in...,N/A,4.4,4.4
1,Renaissance,"Cairo, Renaissance Cairo Mirage City Hotel",https://renaissance-hotels.marriott.com/renais...,"Ring Road, Mirage City, P.O. Box 250, Heliopolis",Cairo,,11757,https://www.marriott.com/hotels/maps/travel/ca...,30.074825,31.439386,N/A,[ Experience 5-star service at Renaissance Cai...,N/A,4.6,4.6
2,Renaissance,"Sharm El Sheikh, Renaissance Sharm El Sheikh G...",https://renaissance-hotels.marriott.com/renais...,"Al Fanar Street, Om El Seid Hill",Sharm El Sheikh,,,Reviews,https://www.marriott.co.uk/hotels/maps/travel/...,27.8517,34.31304,[ Overlooking Tiran Island -one of the world's...,N/A,4.5,4.4
3,Renaissance,"Beijing, Renaissance Beijing Capital Hotel",https://renaissance-hotels.marriott.com/renais...,"61 Dongsanhuan Middle Road, Chaoyang District",Beijing,,100022,Reviews,https://www.marriott.com/hotels/maps/travel/bj...,39.897967,116.460698,"[ Inspiring stories, discovery for yourself. S...",N/A,4.7,4.7
4,Renaissance,"Beijing, Renaissance Beijing Wangfujing Hotel",https://renaissance-hotels.marriott.com/renais...,"57 Wangfujing Street, Dongcheng District",Beijing,,100006,https://www.marriott.com/hotels/maps/travel/bj...,39.921223,116.41007,N/A,[ Deftly combining traditional elegance and mo...,N/A,4.7,4.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166,Renaissance,"Fort Worth, The Worthington Renaissance Fort W...",https://renaissance-hotels.marriott.com/worthi...,200 Main Street,Fort Worth,Texas,76102,Reviews,https://www.marriott.com/hotels/maps/travel/df...,32.756443,-97.332436,[ A historic fixture on the downtown landscape...,N/A,4.6,4.5
167,Renaissance,"Richardson, Renaissance Dallas Richardson Hotel",https://renaissance-hotels.marriott.com/renais...,900 E. Lookout Drive,Richardson,Texas,75082,Reviews,https://www.marriott.com/hotels/maps/travel/da...,32.986887,-96.710049,[ Stay productive at Renaissance Dallas Richar...,N/A,4.5,4.4
168,Renaissance,"Arlington, Renaissance Arlington Capital View ...",https://renaissance-hotels.marriott.com/renais...,2800 South Potomac Ave,Arlington,Virginia,22202,https://www.marriott.com/hotels/maps/travel/wa...,38.84763,-77.050953,N/A,[ Discover LEED® Gold certified accommodations...,N/A,4.8,4.7
169,Renaissance,"Portsmouth, Renaissance Portsmouth-Norfolk Wat...",https://renaissance-hotels.marriott.com/renais...,425 Water St,Portsmouth,Virginia,23704,Reviews,https://www.marriott.com/hotels/maps/travel/or...,36.837116,-76.296888,[ You will find the Renaissance Portsmouth-Nor...,N/A,4.4,4.4


In [6]:
hotel_df = hotel_df.drop_duplicates(subset='name')

In [7]:
hotel_df

,brand,name,url,street,locality,state,postalcode,coordinate1,coordinate2,coordinate3,coordinate4,description,overall_rating,cleanliness_rating,service_rating
0,Renaissance,"Tlemcen, Renaissance Tlemcen Hotel",https://renaissance-hotels.marriott.com/renais...,Plateau Lalla Setti,Tlemcen,,13000,Reviews,https://www.marriott.com/hotels/maps/travel/al...,34.867492,-1.315989,[ The Renaissance Tlemcen is a 5-star hotel in...,N/A,4.4,4.4
1,Renaissance,"Cairo, Renaissance Cairo Mirage City Hotel",https://renaissance-hotels.marriott.com/renais...,"Ring Road, Mirage City, P.O. Box 250, Heliopolis",Cairo,,11757,https://www.marriott.com/hotels/maps/travel/ca...,30.074825,31.439386,N/A,[ Experience 5-star service at Renaissance Cai...,N/A,4.6,4.6
2,Renaissance,"Sharm El Sheikh, Renaissance Sharm El Sheikh G...",https://renaissance-hotels.marriott.com/renais...,"Al Fanar Street, Om El Seid Hill",Sharm El Sheikh,,,Reviews,https://www.marriott.co.uk/hotels/maps/travel/...,27.8517,34.31304,[ Overlooking Tiran Island -one of the world's...,N/A,4.5,4.4
3,Renaissance,"Beijing, Renaissance Beijing Capital Hotel",https://renaissance-hotels.marriott.com/renais...,"61 Dongsanhuan Middle Road, Chaoyang District",Beijing,,100022,Reviews,https://www.marriott.com/hotels/maps/travel/bj...,39.897967,116.460698,"[ Inspiring stories, discovery for yourself. S...",N/A,4.7,4.7
4,Renaissance,"Beijing, Renaissance Beijing Wangfujing Hotel",https://renaissance-hotels.marriott.com/renais...,"57 Wangfujing Street, Dongcheng District",Beijing,,100006,https://www.marriott.com/hotels/maps/travel/bj...,39.921223,116.41007,N/A,[ Deftly combining traditional elegance and mo...,N/A,4.7,4.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166,Renaissance,"Fort Worth, The Worthington Renaissance Fort W...",https://renaissance-hotels.marriott.com/worthi...,200 Main Street,Fort Worth,Texas,76102,Reviews,https://www.marriott.com/hotels/maps/travel/df...,32.756443,-97.332436,[ A historic fixture on the downtown landscape...,N/A,4.6,4.5
167,Renaissance,"Richardson, Renaissance Dallas Richardson Hotel",https://renaissance-hotels.marriott.com/renais...,900 E. Lookout Drive,Richardson,Texas,75082,Reviews,https://www.marriott.com/hotels/maps/travel/da...,32.986887,-96.710049,[ Stay productive at Renaissance Dallas Richar...,N/A,4.5,4.4
168,Renaissance,"Arlington, Renaissance Arlington Capital View ...",https://renaissance-hotels.marriott.com/renais...,2800 South Potomac Ave,Arlington,Virginia,22202,https://www.marriott.com/hotels/maps/travel/wa...,38.84763,-77.050953,N/A,[ Discover LEED® Gold certified accommodations...,N/A,4.8,4.7
169,Renaissance,"Portsmouth, Renaissance Portsmouth-Norfolk Wat...",https://renaissance-hotels.marriott.com/renais...,425 Water St,Portsmouth,Virginia,23704,Reviews,https://www.marriott.com/hotels/maps/travel/or...,36.837116,-76.296888,[ You will find the Renaissance Portsmouth-Nor...,N/A,4.4,4.4


In [8]:
hotel_df.to_csv('renaissance_description.csv')